Rename notebook content to snake_case
===

In [17]:
import json
import nbformat
import re
from glob import glob
import ast


def snake_case(name):
    """
    Convert CamelCase to snake_case
    """
    s1 = re.sub("(.)([A-Z][a-z]+)", r"\1_\2", name)
    return re.sub("([a-z0-9])([A-Z])", r"\1_\2", s1).lower()


def rename_identifiers(code):
    try:
        tree = ast.parse(code)
        for node in ast.walk(tree):
            if isinstance(node, ast.Name):
                if not node.id.startswith("_"):
                    new_name = snake_case(node.id)
                    code = code.replace(node.id, new_name)
            elif isinstance(node, ast.FunctionDef):
                if not node.name.startswith("_"):
                    new_name = snake_case(node.name)
                    code = code.replace(node.name, new_name)
    except SyntaxError:
        pass  # Skip if there's a SyntaxError (non-Python code)

    return code


notebooks = glob("./*.ipynb")
notebooks = [nb for nb in notebooks if nb != "./to_snake_case.ipynb"]

for notebook_file in notebooks:
    # Read your Jupyter notebook
    with open(notebook_file, "r") as f:
        notebook_content = nbformat.read(f, as_version=4)

    # Iterate through the cells in the notebook
    for cell in notebook_content.cells:
        if cell.cell_type == "code":
            cell.source = rename_identifiers(cell.source)

    # Write back to the notebook
    with open(notebook_file, "w") as f:
        nbformat.write(notebook_content, f)

After this run `black-nb .` to format all notebooks in the directory